Testing derivaties for 1D MT problem.

Especially the rx.projectFieldsDeriv

In [1]:
import SimPEG as simpeg
import simpegEM as simpegem, simpegMT as simpegmt
from SimPEG.Utils import meshTensor
import numpy as np

In [3]:
# Setup the problem
sigmaHalf = 1e-2
# Frequency
nFreq = 33
freqs = np.logspace(3,-3,nFreq)
# Make the mesh
ct = 5
air = meshTensor([(ct,25,1.3)])
# coreT0 = meshTensor([(ct,15,1.2)])
# coreT1 = np.kron(meshTensor([(coreT0[-1],15,1.3)]),np.ones((7,)))
core = np.concatenate( (  np.kron(meshTensor([(ct,15,-1.2)]),np.ones((10,))) , meshTensor([(ct,20)]) ) )
bot = meshTensor([(core[0],10,-1.3)])
x0 = -np.array([np.sum(np.concatenate((core,bot)))])
m1d = simpeg.Mesh.TensorMesh([np.concatenate((bot,core,air))], x0=x0)
# Make the model
sigma = np.zeros(m1d.nC) + sigmaHalf
sigma[ m1d.gridCC > 0 ] = 1e-8

rxList = []
for rxType in ['z1dr','z1di']:
    rxList.append(simpegmt.SurveyMT.RxMT(simpeg.mkvc(np.array([0.0]),2).T,rxType))
# Source list
srcList =[]
tD = False
if tD:
    for freq in freqs:
        srcList.append(simpegmt.SurveyMT.srcMT_polxy_1DhomotD(rxList,freq))
else:
    for freq in freqs:
        srcList.append(simpegmt.SurveyMT.srcMT_polxy_1Dprimary(rxList,freq,sigma))
# Make the survey
survey = simpegmt.SurveyMT.SurveyMT(srcList)

# Set the problem
problem = simpegmt.ProblemMT1D.eForm_psField(m1d)
problem.pair(survey)

# Get the fields
fields = problem.fields(sigma)

# Project the data
data = survey.projectFields(fields)


Project at freq: 1.000e+03
Project at freq: 6.494e+02
Project at freq: 4.217e+02
Project at freq: 2.738e+02
Project at freq: 1.778e+02
Project at freq: 1.155e+02
Project at freq: 7.499e+01
Project at freq: 4.870e+01
Project at freq: 3.162e+01
Project at freq: 2.054e+01
Project at freq: 1.334e+01
Project at freq: 8.660e+00
Project at freq: 5.623e+00
Project at freq: 3.652e+00
Project at freq: 2.371e+00
Project at freq: 1.540e+00
Project at freq: 1.000e+00
Project at freq: 6.494e-01
Project at freq: 4.217e-01
Project at freq: 2.738e-01
Project at freq: 1.778e-01
Project at freq: 1.155e-01
Project at freq: 7.499e-02
Project at freq: 4.870e-02
Project at freq: 3.162e-02
Project at freq: 2.054e-02
Project at freq: 1.334e-02
Project at freq: 8.660e-03
Project at freq: 5.623e-03
Project at freq: 3.652e-03
Project at freq: 2.371e-03
Project at freq: 1.540e-03
Project at freq: 1.000e-03


We need calculate this derivative. 
\begin{align}
\underbrace{\frac{\partial P(f(u(m)),m^{fix})}{\partial f}}_{Rx}
\end{align}

Use the rule
\begin{align}
\frac{d}{dx}\left( \frac{a(x)}{b(x)} \right)  = \frac{\frac{d }{dx} a(x)  b(x) - a(x)\frac{d }{dx} b(x)  }{ b(x)^2 }
\end{align}

In the case of the 1D MT problem the data is calculated as 
\begin{align}
MT1Ddata = P(f(m)) &= \frac{P_{ex} f_e(src,m)}{P_{bx} f_b(src,m) \frac{1}{\mu_0}} = \frac{P_e u}{P_b f_b(u)} \\
\frac{\partial P(f(m))}{\partial u} v &=  \frac{P_e}{P_b \frac{1}{mu_0} f_b(u)}v - \frac{P_e u}{\left(P_b \frac{1}{mu_0} f_b(u)\right)^2} P_b \frac{1}{mu_0} \frac{d f_b}{du} v
\end{align}
where u is the fields that we solve for. 
\begin{align}
\frac{d f_b}{du} = - \frac{1}{i \omega} \nabla 
\end{align}



In [4]:
# Unused code &= \frac{ P_{ex} P_{bx} \frac{1}{\mu_0} \left( f_b(src,m) - f_e(src,m) \right) } { \left(P_{bx}f_b(src,m) \frac{1}{\mu_0} \right)^2 }

As matrices the formulas above can be written as
\begin{align}
\left[ \frac{\partial P(f(m))}{\partial u} v \right] = diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right] [P_e v] - diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right]^T diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right] \left[ P_b   \frac{d f_b}{du}(v) \frac{1}{mu_0} \right]
\end{align}



The adjoint problem is done simliarly
\begin{align}
\left[ \frac{\partial P(f(m))}{\partial u} v \right]^T = [P_e v]^T diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right]^T  - \left[ P_b   \frac{d f_b}{du}(v) \frac{1}{mu_0} \right]^T diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right] diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right]^T 
\end{align}


In [6]:
# def projectFields(self, src, mesh, u):
#         '''
#         Project the fields and return the
#         '''

#         if self.projType is 'Z1D':
#             Pex = mesh.getInterpolationMat(self.locs,'Fx')
#             Pbx = mesh.getInterpolationMat(self.locs,'Ex')
#             ex = Pex*mkvc(u[src,'e_1d'],2)
#             bx = Pbx*mkvc(u[src,'b_1d'],2)/mu_0
#             f_part_complex = ex/bx
#         real_or_imag = self.projComp
#         f_part = getattr(f_part_complex, real_or_imag)
#         return f_part

In [7]:
# Initate things for the derivs Test
src = survey.srcList[0]
rx = src.rxList[0]
u0 = np.random.randn(m1d.nN)+np.random.randn(m1d.nN)*1j
f0 = problem.fieldsPair(m1d,survey)
f0[src,'e_1d'] = u0
f0[src,'b_1d'] = -1/(1j*simpegem.Utils.EMUtils.omega(src.freq))*m1d.nodalGrad*u0

In [8]:
u0

array([  1.31527642e-01+1.45769269j,   5.95564562e-01+0.42028906j,
        -9.59349955e-01-1.11757166j,  -1.40416822e+00-0.41446777j,
         3.13493260e+00+0.03428491j,  -1.51344609e+00-1.00877232j,
         3.24356486e-01+0.65648507j,   2.67095094e-01+1.13805012j,
         1.73664285e+00-0.65728832j,  -8.97426739e-01+0.2288435j ,
         4.80610771e-01+0.49953842j,   5.13958825e-01+1.0780965j ,
        -4.99094522e-01-1.4264564j ,   1.70175950e-01-1.02378638j,
        -7.24199015e-01-0.8156263j ,  -1.80125549e+00+0.24188448j,
        -1.34681899e+00+0.30820186j,   5.48408346e-01-0.9591145j ,
         3.94350988e-01+0.82075162j,  -4.82329564e-01-0.0788764j ,
         1.86240416e+00+0.52052826j,  -1.29718885e+00+1.18797306j,
        -2.97912489e-01+1.46517189j,   3.70500990e-01-1.38665192j,
         6.63769486e-01-0.37204627j,  -6.73249451e-01+0.20464947j,
         8.01428477e-01-0.04983355j,  -3.24457843e-01-1.19101563j,
         7.19176651e-01-0.65664744j,   1.15011866e+00-0.322104

In [10]:
# Run a test
def fun(u):
    f = problem.fieldsPair(m1d,survey)
    f[src,'e_1d'] = u
    f[src,'b_1d'] = -(m1d.nodalGrad*u)/(1j*simpegem.Utils.EMUtils.omega(src.freq))
    return rx.projectFields(src,m1d,f), lambda t: rx.projectFieldsDeriv(src,m1d,f0,t)
simpeg.Tests.checkDerivative(fun,u0,num=3,plotIt=False)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    6.154e-03     8.309e-03      nan
 1   1.00e-02    5.944e-04     8.099e-04      1.011
 2   1.00e-03    5.915e-05     8.070e-05      1.002
*********************************************************
<<<<<<<<<<<<<<<<<<<<<<<<< FAIL! >>>>>>>>>>>>>>>>>>>>>>>>>
*********************************************************
Did you put your clever trousers on today?



False

In [ ]:
print rx.projectFieldsDeriv(src,m1d,f0,u0)
print m1d.nF
print m1d.nN

In [ ]:
fields._b_1dDeriv_u(src,u0)

In [ ]:
%debug